In [ ]:
import os

PATH = '/home/kai/data/kaggle/talkingdata/haoyan_data/'
cols = ['os', 'hour', 'device', 'channel', 'app']
nrows = 10
train = pd.read_feather(PATH + 'train_cleaned.ftr', usecols=cols, nrows=nrows)
test = pd.read_feather(PATH + 'test_cleaned.ftr', usecols=cols, nrows=nrows)
for file in os.listdir(PATH):
    tmp = file.split('.')
    if tmp[0] != 'train_cleaned' or tmp[0] != 'test_cleaned' and tmp[1] == 'ftr':
        ttmp = tmp[0].split('_')
        if ttmp[0] == 'train': train[ttmp[1]] = pd.read_feather(PATH + file, df_len=len(train))
        else: test[ttmp[1]] == pd.read_feather(PATH + file, df_len=len(test))